**Running fcn8**

In [12]:
import numpy as np
import fcn8
reload(fcn8)
from fcn8 import Fcn8

print_summary = False

# load an image
IMAGE_SIZE = 224
img_shape = (1, IMAGE_SIZE, IMAGE_SIZE, 3) # tf format (N,H,W,C)
img = np.linspace(-0.1, 0.5, num=np.prod(img_shape)).reshape(img_shape)

# running on one batch
fcn = Fcn8()
res = fcn.predict(img)
print res.shape

model = fcn.getModel()
if print_summary: 
    model.summary()



(1, 7, 7, 1000)


There are some posibilities for fine tuning a 2-class convnet:
Here, I'll try one possibility:
1. pop the last Conv2D layer (None, 7, 7, 1000).
2. set all layers to NOT TRAINABLE.
3. Add a fine tune Conv2D layer of two classes (None, 7, 7, 2). Then train only this layer.

In order to train the last (added) 2-class layers, I just add a dense layer at the end with softmax activation.

So, let's do this.

In [13]:
# pop out last layer
model.pop()
if print_summary:
    model.summary()

In [14]:
# set all layers to not trainable
for layer in model.layers: layer.trainable = False
if print_summary:
    model.summary()

In [15]:
# add a flatten layer
from keras.layers.convolutional import Convolution2D

model.add(Convolution2D(2, 7, 7, border_mode = 'same'))
if print_summary:
    model.summary()

In [16]:
# add dense layer from (None, 200704) ---> (None, 2)
from keras.layers.core import Dense, Flatten

# first flatten the last layer
model.add(Flatten())
# then input it to a dense layer
model.add(Dense(2, activation='softmax'))
if True: 
    model.summary()


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_5 (Lambda)                (None, 224, 224, 3)   0           lambda_input_5[0][0]             
____________________________________________________________________________________________________
convolution2d_67 (Convolution2D) (None, 224, 224, 64)  0           lambda_5[0][0]                   
____________________________________________________________________________________________________
convolution2d_68 (Convolution2D) (None, 224, 224, 64)  0           convolution2d_67[0][0]           
____________________________________________________________________________________________________
maxpooling2d_21 (MaxPooling2D)   (None, 112, 112, 64)  0           convolution2d_68[0][0]           
___________________________________________________________________________________________

from here what we have to do is to learn the parametters - they are 
Once this is done, what we have to do is to remove the last two layers dense_2 and flatten_2 and continue the flow of fcn8.
some notes:
1. here the params of convolution2d_83 are NOT initialized. it may be interesting to initialize them using 
   the scheme describe in https://github.com/MarvinTeichmann/tensorflow-fcn/blob/master/fcn8_vgg.py
   and check if the training is improving (accuracy + speed) or not.